In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
df = pd.read_csv('./data.csv', sep='|')
df.head()

,client_id,Target,Month,First_product_dt,Last_product_dt,CreditCard_Premium,CreditCard_Active,CreditCard_CoBranding,Loan_Active,Mortgage_Active,...,CreditCard_Payment_External,CreditCard_Payment_Cash,CreditCard_Payment_Web,CreditCard_Payment_ATM,CreditCard_Payment_TAS,Investment_Numbers,Mobile,Email,Region,CreditCard_Product
0,5856970,1.0,2018-10-01,2013-10-23,2019-01-10,No,Yes,No,No,No,...,0.0,0.0,0.0,0.0,0.0,1.0,Yes,Yes,NaN,NaN
1,6371753,0.0,2018-09-01,2015-07-29,2018-06-02,No,No,No,No,No,...,0.0,0.0,0.0,0.0,0.0,0.0,Yes,No,NaN,NaN
2,5928737,0.0,2019-01-01,2016-08-31,2018-12-27,No,No,No,No,No,...,0.0,0.0,0.0,0.0,0.0,0.0,Yes,Yes,NaN,NaN
3,475064,0.0,2018-12-01,2014-07-13,2017-11-30,No,Yes,No,No,No,...,0.0,0.0,0.0,0.0,0.0,0.0,Yes,Yes,NaN,NaN
4,3615172,0.0,2018-09-01,2017-12-27,2017-12-28,No,No,No,No,No,...,0.0,0.0,0.0,0.0,0.0,0.0,Yes,No,NaN,NaN


In [3]:
def toDatetime(column):
    return pd.to_datetime(df[column], format="%Y-%m-%d")

In [4]:
# Replace dates columns with datetime types instead of "object"
df['Month'] = toDatetime('Month')
df['First_product_dt'] = toDatetime('First_product_dt')
df['Last_product_dt'] = toDatetime('Last_product_dt')

### Number of columns

In [5]:
len(df.columns)

77

### Number of rows

In [6]:
len(df)

238615

### Number of columns related to saving account

In [7]:
saving_account_columns = [col for col in df.columns if col.startswith('SavingAccount_')]
len(saving_account_columns)

29

### Number of months

In [8]:
months = pd.unique(df[df['Month'].notnull()]['Month'])
len(months)

9

### Number of clients

In [9]:
clients = pd.unique(df[df['client_id'].notnull()]['client_id'])
len(clients)

26560